`text2vec-transformers` is **only** available through Weaviate open-source. Here are options to select your desired model: 

1. [Pre-built transformers model containers](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#pre-built-images)

2. [Any model from Hugging Face Model Hub](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#option-2-use-any-publicly-available-hugging-face-model)

3. [Use any private or local PyTorch or Tensorflow transformer model](https://weaviate.io/developers/weaviate/modules/retriever-vectorizer-modules/text2vec-transformers#option-3-custom-build-with-a-private-or-local-model)

## Schema

In [ ]:
import weaviate

client = weaviate.Client("http://localhost:8080")

# resetting the schema. CAUTION: THIS WILL DELETE YOUR DATA 
client.schema.delete_all()

schema = {
   "classes": [
       {
           "class": "JeopardyQuestion",
           "description": "List of jeopardy questions",
           "moduleConfig": { # specify the vectorizer and model type you're using
               "text2vec-transformers": { 
                    "poolingStrategy": "masked_mean"
                }
           },
           "properties": [
               {
                   "name": "Category",
                   "dataType": ["text"],
                   "description": "Category of the question",
               },
               {
                "name": "Question",
                "dataType": ["text"],
                "description": "The question",
               },
               {
                   "name": "Answer",
                   "dataType": ["text"],
                   "description": "The answer",
                }
            ]
        }
    ]
}

client.schema.create(schema)

print("Successfully created the schema.")

## Import the Data

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

if client.is_ready():

# Configure a batch process
  with client.batch as batch:
      batch.batch_size=100
      # Batch import all Questions
      for i, d in enumerate(data):
          print(f"importing question: {i+1}")

          properties = {
              "answer": d["Answer"],
              "question": d["Question"],
              "category": d["Category"],
          }

          client.batch.add_data_object(properties, "JeopardyQuestion")
else:
  print("The Weaviate cluster is not connected.")

## Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Hybrid Search only

The below query is finding Jeopardy questions about animals and is limiting the output to only two results. Notice `alpha` is set to `0.80`, which means it is weighing the vector search results more than bm25. If you were to set `alpha = 0.25`, you would get different results. 

In [ ]:
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_hybrid(
        query = "animal",
        alpha = 0.80
    )
    .with_limit(3)
    .do()
)

print(json.dumps(response, indent=2))

### Hybrid Search on a specific property

The `properties` parameter allows you to list the properties that you want bm25 to search on.

In [ ]:
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_hybrid(
        query = "animal",
        properties = ["question"],
        alpha = 0.80
    )
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=2))

### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [ ]:
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_hybrid(
        query = "elephant",
        alpha = 0.5
    )
    .with_where({
        "path": ["category"],
        "operator": "Equal",
        "valueText": "Animals"
    })
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=2))

### Hybrid Search with a custom vector

You can pass in your own vector as input into the hybrid query, by using the `vector` parameter. 

In [ ]:
vector = [-0.0125526935, -0.021168863, ...]

response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_hybrid(
        query = "animal",
        vector = vector
    )
    .with_limit(2)
    .do()
)

print(json.dumps(response, indent=2))